In [1]:
import pandas as pd
import numpy as np

def create_outlet_size_imputer(train_data):
    """
    Create outlet size imputation mapping from training data.
    
    Parameters:
    train_data (pd.DataFrame): Training dataset
    
    Returns:
    dict: Imputation mapping for outlet sizes
    """
    # Calculate mode for each combination of Outlet_Location_Type and Outlet_Type
    outlet_size_mode = train_data.groupby(['Outlet_Location_Type', 'Outlet_Type'])['Outlet_Size'].apply(
        lambda x: x.mode()[0] if not x.mode().empty else 'Small'
    ).to_dict()
    
    print("Outlet Size Imputation Mapping:")
    for key, value in outlet_size_mode.items():
        print(f"  {key}: {value}")
    
    return outlet_size_mode

def impute_outlet_size(data, outlet_size_mapping):
    """
    Impute outlet sizes using pre-computed mapping.
    
    Parameters:
    data (pd.DataFrame): Dataset to impute
    outlet_size_mapping (dict): Mapping from create_outlet_size_imputer
    
    Returns:
    pd.DataFrame: Dataset with imputed outlet sizes
    """
    data_copy = data.copy()
    
    # Count missing values before imputation
    missing_before = data_copy['Outlet_Size'].isnull().sum()
    print(f"Missing Outlet_Size before imputation: {missing_before}")
    
    # Apply imputation
    data_copy['Outlet_Size'] = data_copy.apply(
        lambda row: outlet_size_mapping.get(
            (row['Outlet_Location_Type'], row['Outlet_Type']), 'Small'
        ) if pd.isna(row['Outlet_Size']) else row['Outlet_Size'], 
        axis=1
    )
    
    # Count missing values after imputation
    missing_after = data_copy['Outlet_Size'].isnull().sum()
    print(f"Missing Outlet_Size after imputation: {missing_after}")
    
    return data_copy

def create_item_weight_imputer(train_data):
    """
    Create item weight imputation mappings from training data.
    Per-unit cost mapping is calculated after applying previous imputation steps.
    
    Parameters:
    train_data (pd.DataFrame): Training dataset
    
    Returns:
    dict: Dictionary containing all imputation mappings
    """
    print("Creating item weight imputation mappings...")
    
    # Make a copy to avoid modifying original data
    train_copy = train_data.copy()
    
    # Step 1: Create median MRP mappings
    median_mrp_1 = train_copy.pivot_table(
        index=["Item_Identifier", "Item_Type", "Outlet_Type"],
        values="Item_MRP", 
        aggfunc=np.nanmedian
    ).reset_index().rename(columns={"Item_MRP": "Median_Item_MRP"})
    
    median_mrp_2 = train_copy.pivot_table(
        index=["Item_Type", "Outlet_Type"],
        values="Item_MRP", 
        aggfunc=np.nanmedian
    ).reset_index().rename(columns={"Item_MRP": "Median_Item_MRP_1"})
    
    # Add median MRP columns to training data copy
    train_copy = train_copy.merge(median_mrp_1, on=["Item_Identifier", "Item_Type", "Outlet_Type"], how="left")
    train_copy = train_copy.merge(median_mrp_2, on=["Item_Type", "Outlet_Type"], how="left")
    
    # Step 2: Create weight mappings at different granularities (using original data)
    # Level 1: Item_Identifier + Outlet_Type + Median_MRP
    weight_mapping_1 = {}
    weight_stats_1 = train_copy.pivot_table(
        index=["Item_Identifier", "Outlet_Type", "Median_Item_MRP"],
        values="Item_Weight",
        aggfunc=np.nanmean
    ).reset_index()
    
    for _, row in weight_stats_1.iterrows():
        if not pd.isna(row['Item_Weight']):
            key = (row['Item_Identifier'], row['Outlet_Type'], row['Median_Item_MRP'])
            weight_mapping_1[key] = row['Item_Weight']
    
    # Level 2: Item_Identifier + Item_MRP
    weight_mapping_2 = {}
    weight_stats_2 = train_copy.pivot_table(
        index=["Item_Identifier", "Item_MRP"],
        values="Item_Weight",
        aggfunc=np.nanmean
    ).reset_index()
    
    for _, row in weight_stats_2.iterrows():
        if not pd.isna(row['Item_Weight']):
            key = (row['Item_Identifier'], row['Item_MRP'])
            weight_mapping_2[key] = row['Item_Weight']
    
    # Level 3: Item_Identifier only
    weight_mapping_3 = {}
    weight_stats_3 = train_copy.pivot_table(
        index=["Item_Identifier"],
        values="Item_Weight",
        aggfunc=np.nanmean
    ).reset_index()
    
    for _, row in weight_stats_3.iterrows():
        if not pd.isna(row['Item_Weight']):
            weight_mapping_3[row['Item_Identifier']] = row['Item_Weight']
    
    # Level 4: Item_Type + Median_MRP_1
    weight_mapping_4 = {}
    weight_stats_4 = train_copy.pivot_table(
        index=["Item_Type", "Median_Item_MRP_1"],
        values="Item_Weight",
        aggfunc=np.nanmean
    ).reset_index()
    
    for _, row in weight_stats_4.iterrows():
        if not pd.isna(row['Item_Weight']):
            key = (row['Item_Type'], row['Median_Item_MRP_1'])
            weight_mapping_4[key] = row['Item_Weight']
    
    # Step 3: Apply previous imputation steps to get more complete weight data
    print("Applying previous imputation steps to calculate per-unit cost...")
    
    # Apply Level 1-4 imputation to training data copy
    missing_before_cost_calc = train_copy['Item_Weight'].isnull().sum()
    
    # Level 1 imputation
    mask_1 = train_copy['Item_Weight'].isnull()
    for idx in train_copy[mask_1].index:
        row = train_copy.loc[idx]
        key = (row['Item_Identifier'], row['Outlet_Type'], row['Median_Item_MRP'])
        if key in weight_mapping_1:
            train_copy.loc[idx, 'Item_Weight'] = weight_mapping_1[key]
    
    # Level 2 imputation
    mask_2 = train_copy['Item_Weight'].isnull()
    for idx in train_copy[mask_2].index:
        row = train_copy.loc[idx]
        key = (row['Item_Identifier'], row['Item_MRP'])
        if key in weight_mapping_2:
            train_copy.loc[idx, 'Item_Weight'] = weight_mapping_2[key]
    
    # Level 3 imputation
    mask_3 = train_copy['Item_Weight'].isnull()
    for idx in train_copy[mask_3].index:
        row = train_copy.loc[idx]
        if row['Item_Identifier'] in weight_mapping_3:
            train_copy.loc[idx, 'Item_Weight'] = weight_mapping_3[row['Item_Identifier']]
    
    # Level 4 imputation
    mask_4 = train_copy['Item_Weight'].isnull()
    for idx in train_copy[mask_4].index:
        row = train_copy.loc[idx]
        key = (row['Item_Type'], row['Median_Item_MRP_1'])
        if key in weight_mapping_4:
            train_copy.loc[idx, 'Item_Weight'] = weight_mapping_4[key]
    
    missing_after_previous_steps = train_copy['Item_Weight'].isnull().sum()
    print(f"Missing weights before cost calculation: {missing_before_cost_calc}")
    print(f"Missing weights after previous steps: {missing_after_previous_steps}")
    print(f"Additional weights filled for cost calculation: {missing_before_cost_calc - missing_after_previous_steps}")
    
    # Step 4: Now calculate per-unit cost using the more complete weight data
    train_copy["perUnitCost"] = train_copy["Item_Weight"] / train_copy["Item_MRP"]
    
    per_unit_cost_mapping = {}
    cost_stats = train_copy.pivot_table(
        index=["Item_Type", "Outlet_Type"],
        values="perUnitCost",
        aggfunc=np.nanmean
    ).reset_index()
    
    for _, row in cost_stats.iterrows():
        if not pd.isna(row['perUnitCost']):
            key = (row['Item_Type'], row['Outlet_Type'])
            per_unit_cost_mapping[key] = row['perUnitCost']
    
    print(f"Created {len(weight_mapping_1)} Level-1 mappings (Item+Outlet+MRP)")
    print(f"Created {len(weight_mapping_2)} Level-2 mappings (Item+MRP)")
    print(f"Created {len(weight_mapping_3)} Level-3 mappings (Item only)")
    print(f"Created {len(weight_mapping_4)} Level-4 mappings (Type+MRP)")
    print(f"Created {len(per_unit_cost_mapping)} per-unit cost mappings (calculated after previous imputation)")
    
    return {
        'median_mrp_1': median_mrp_1,
        'median_mrp_2': median_mrp_2,
        'weight_mapping_1': weight_mapping_1,
        'weight_mapping_2': weight_mapping_2,
        'weight_mapping_3': weight_mapping_3,
        'weight_mapping_4': weight_mapping_4,
        'per_unit_cost_mapping': per_unit_cost_mapping
    }

def impute_item_weight(data, weight_imputation_mappings):
    """
    Impute item weights using pre-computed mappings.
    
    Parameters:
    data (pd.DataFrame): Dataset to impute
    weight_imputation_mappings (dict): Mappings from create_item_weight_imputer
    
    Returns:
    pd.DataFrame: Dataset with imputed item weights
    """
    data_copy = data.copy()
    
    # Count missing values before imputation
    missing_before = data_copy['Item_Weight'].isnull().sum()
    print(f"Missing Item_Weight before imputation: {missing_before}")
    
    # Extract mappings
    median_mrp_1 = weight_imputation_mappings['median_mrp_1']
    median_mrp_2 = weight_imputation_mappings['median_mrp_2']
    weight_mapping_1 = weight_imputation_mappings['weight_mapping_1']
    weight_mapping_2 = weight_imputation_mappings['weight_mapping_2']
    weight_mapping_3 = weight_imputation_mappings['weight_mapping_3']
    weight_mapping_4 = weight_imputation_mappings['weight_mapping_4']
    per_unit_cost_mapping = weight_imputation_mappings['per_unit_cost_mapping']
    
    # Add median MRP columns
    data_copy = data_copy.merge(median_mrp_1, on=["Item_Identifier", "Item_Type", "Outlet_Type"], how="left")
    data_copy = data_copy.merge(median_mrp_2, on=["Item_Type", "Outlet_Type"], how="left")
    
    # Track imputation progress
    missing_counts = [data_copy['Item_Weight'].isnull().sum()]
    
    # Level 1 imputation: Item_Identifier + Outlet_Type + Median_MRP
    mask_1 = data_copy['Item_Weight'].isnull()
    for idx in data_copy[mask_1].index:
        row = data_copy.loc[idx]
        key = (row['Item_Identifier'], row['Outlet_Type'], row['Median_Item_MRP'])
        if key in weight_mapping_1:
            data_copy.loc[idx, 'Item_Weight'] = weight_mapping_1[key]
    
    missing_counts.append(data_copy['Item_Weight'].isnull().sum())
    print(f"After Level-1 imputation: {missing_counts[-1]} missing")
    
    # Level 2 imputation: Item_Identifier + Item_MRP
    mask_2 = data_copy['Item_Weight'].isnull()
    for idx in data_copy[mask_2].index:
        row = data_copy.loc[idx]
        key = (row['Item_Identifier'], row['Item_MRP'])
        if key in weight_mapping_2:
            data_copy.loc[idx, 'Item_Weight'] = weight_mapping_2[key]
    
    missing_counts.append(data_copy['Item_Weight'].isnull().sum())
    print(f"After Level-2 imputation: {missing_counts[-1]} missing")
    
    # Level 3 imputation: Item_Identifier only
    mask_3 = data_copy['Item_Weight'].isnull()
    for idx in data_copy[mask_3].index:
        row = data_copy.loc[idx]
        if row['Item_Identifier'] in weight_mapping_3:
            data_copy.loc[idx, 'Item_Weight'] = weight_mapping_3[row['Item_Identifier']]
    
    missing_counts.append(data_copy['Item_Weight'].isnull().sum())
    print(f"After Level-3 imputation: {missing_counts[-1]} missing")
    
    # Level 4 imputation: Item_Type + Median_MRP_1
    mask_4 = data_copy['Item_Weight'].isnull()
    for idx in data_copy[mask_4].index:
        row = data_copy.loc[idx]
        key = (row['Item_Type'], row['Median_Item_MRP_1'])
        if key in weight_mapping_4:
            data_copy.loc[idx, 'Item_Weight'] = weight_mapping_4[key]
    
    missing_counts.append(data_copy['Item_Weight'].isnull().sum())
    print(f"After Level-4 imputation: {missing_counts[-1]} missing")
    
    # Level 5 imputation: Per-unit cost approach
    mask_5 = data_copy['Item_Weight'].isnull()
    for idx in data_copy[mask_5].index:
        row = data_copy.loc[idx]
        key = (row['Item_Type'], row['Outlet_Type'])
        if key in per_unit_cost_mapping:
            estimated_weight = row['Item_MRP'] * per_unit_cost_mapping[key]
            data_copy.loc[idx, 'Item_Weight'] = estimated_weight
    
    missing_counts.append(data_copy['Item_Weight'].isnull().sum())
    print(f"After Level-5 imputation: {missing_counts[-1]} missing")
    
    # Clean up temporary columns
    columns_to_drop = ['Median_Item_MRP', 'Median_Item_MRP_1']
    data_copy.drop(columns=[col for col in columns_to_drop if col in data_copy.columns], inplace=True)
    
    # Final missing count
    missing_after = data_copy['Item_Weight'].isnull().sum()
    print(f"Final missing Item_Weight: {missing_after}")
    print(f"Successfully imputed {missing_before - missing_after} values")
    
    return data_copy

In [2]:

datPath = "/Users/whysocurious/Documents/MLDSAIProjects/SalesPred_Hackathon/data/raw/"

train = pd.read_csv(datPath+'train_v9rqX0R.csv')
test = pd.read_csv(datPath+'test_AbJTz2l.csv')

# Step 1: Create imputation mappings from training data
outlet_size_mapping = create_outlet_size_imputer(train)
weight_imputation_mappings = create_item_weight_imputer(train)

# Step 2: Apply to training data
train_imputed = impute_outlet_size(train, outlet_size_mapping)
train_imputed = impute_item_weight(train_imputed, weight_imputation_mappings)

# Step 3: Apply same mappings to test data (no leakage!)
test_imputed = impute_outlet_size(test, outlet_size_mapping)
test_imputed = impute_item_weight(test_imputed, weight_imputation_mappings)

Outlet Size Imputation Mapping:
  ('Tier 1', 'Grocery Store'): Small
  ('Tier 1', 'Supermarket Type1'): Medium
  ('Tier 2', 'Supermarket Type1'): Small
  ('Tier 3', 'Grocery Store'): Small
  ('Tier 3', 'Supermarket Type1'): High
  ('Tier 3', 'Supermarket Type2'): Medium
  ('Tier 3', 'Supermarket Type3'): Medium
Creating item weight imputation mappings...


/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_12716/473157851.py:73: FutureWarning: The provided callable <function nanmedian at 0x1093fc9d0> is currently using DataFrameGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  median_mrp_1 = train_copy.pivot_table(
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_12716/473157851.py:79: FutureWarning: The provided callable <function nanmedian at 0x1093fc9d0> is currently using DataFrameGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  median_mrp_2 = train_copy.pivot_table(
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_12716/473157851.py:92: FutureWarning: The provided callable <function nanmean at 0x1093fc700> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable wi

Applying previous imputation steps to calculate per-unit cost...
Missing weights before cost calculation: 1463
Missing weights after previous steps: 3
Additional weights filled for cost calculation: 1460
Created 3033 Level-1 mappings (Item+Outlet+MRP)
Created 6716 Level-2 mappings (Item+MRP)
Created 1555 Level-3 mappings (Item only)
Created 48 Level-4 mappings (Type+MRP)
Created 64 per-unit cost mappings (calculated after previous imputation)
Missing Outlet_Size before imputation: 2410


/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_12716/473157851.py:187: FutureWarning: The provided callable <function nanmean at 0x1093fc700> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  cost_stats = train_copy.pivot_table(


Missing Outlet_Size after imputation: 0
Missing Item_Weight before imputation: 1463
After Level-1 imputation: 1281 missing
After Level-2 imputation: 1146 missing
After Level-3 imputation: 4 missing
After Level-4 imputation: 3 missing
After Level-5 imputation: 0 missing
Final missing Item_Weight: 0
Successfully imputed 1463 values
Missing Outlet_Size before imputation: 1606
Missing Outlet_Size after imputation: 0
Missing Item_Weight before imputation: 976
After Level-1 imputation: 837 missing
After Level-2 imputation: 759 missing
After Level-3 imputation: 1 missing
After Level-4 imputation: 1 missing
After Level-5 imputation: 0 missing
Final missing Item_Weight: 0
Successfully imputed 976 values


In [3]:
# 3. Clean Item_Fat_Content
print("\n3. Cleaning Item_Fat_Content...")
print("Original values:", train_imputed['Item_Fat_Content'].value_counts())

# Standardize the values
fat_content_mapping = {
    'LF': 'Low Fat',
    'low fat': 'Low Fat', 
    'reg': 'Regular'
}

train_imputed['Item_Fat_Content'] = train_imputed['Item_Fat_Content'].replace(fat_content_mapping)
print("After cleaning:", train_imputed['Item_Fat_Content'].value_counts())

# Create binary encoding
train_imputed['Low_Fat_Flag'] = (train_imputed['Item_Fat_Content'] == 'Low Fat').astype(int)


3. Cleaning Item_Fat_Content...
Original values: Item_Fat_Content
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: count, dtype: int64
After cleaning: Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64


In [4]:
# 4. Handle Item_Visibility zeros
print("\n4. Handling Item_Visibility zeros...")
zero_visibility = (train_imputed['Item_Visibility'] == 0).sum()
print(f"Zero visibility items: {zero_visibility}")


if zero_visibility > 0:
    # Replace zeros with mean visibility by Item_Type
    item_type_mean_visibility = train_imputed[train_imputed['Item_Visibility'] > 0].groupby(['Item_Type','Outlet_Type'])['Item_Visibility'].mean()
     
    train_imputed['Item_Visibility'] = train_imputed.apply(lambda row: item_type_mean_visibility[row['Item_Type'],row['Outlet_Type']] if row['Item_Visibility'] == 0 else row['Item_Visibility'], axis=1)

print(f"Zero visibility after treatment: {(train_imputed['Item_Visibility'] == 0).sum()}")




4. Handling Item_Visibility zeros...
Zero visibility items: 526
Zero visibility after treatment: 0


In [10]:
print("\n5. Target variable transformation...")
# train_mask = train_imputed['source'] == 'train'
original_sales = train_imputed.loc[:, 'Item_Outlet_Sales'].copy()

# Apply log transformation to reduce skewness
train_imputed.loc[:, 'log_sales'] = np.log1p(train_imputed.loc[:, 'Item_Outlet_Sales'])

log_skewness = train_imputed.loc[:, 'log_sales'].skew()
print(f"Original sales skewness: {original_sales.skew():.3f}")
print(f"Log sales skewness: {log_skewness:.3f}")

print("\n=== Data Cleaning Summary ===")
print("Missing values after cleaning:")
print(train_imputed.isnull().sum()[train_imputed.isnull().sum() > 0])

print("\nDataset ready for feature engineering!")
print(f"Final combined shape: {train_imputed.shape}")


5. Target variable transformation...
Original sales skewness: 1.178
Log sales skewness: -0.882

=== Data Cleaning Summary ===
Missing values after cleaning:
Series([], dtype: int64)

Dataset ready for feature engineering!
Final combined shape: (8523, 14)


In [14]:
train_imputed.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales', 'Low_Fat_Flag', 'log_sales'],
      dtype='object')

In [12]:
combined = train_imputed.copy()
print (combined.shape)

(8523, 14)


In [13]:
# Continuing from Phase 1 - Feature Engineering
print("=== Phase 2: Feature Engineering ===")

# 2.1 Basic Features
print("\n2.1 Creating basic derived features...")

# Outlet Age - key feature as older stores might have established customer base
combined['Outlet_Age'] = 2013 - combined['Outlet_Establishment_Year']
combined['Outlet_Age_Squared'] = combined['Outlet_Age'] ** 2

print(f"Outlet age range: {combined['Outlet_Age'].min()} to {combined['Outlet_Age'].max()} years")

# Item MRP bins - price segments often drive different buying behaviors
combined['Item_MRP_Bins'] = pd.cut(
    combined['Item_MRP'], 
    bins=[0, 50, 100, 150, 200, 300], 
    labels=['Very_Low', 'Low', 'Medium', 'High', 'Premium'],
    include_lowest=True
)

print("MRP distribution across bins:")
print(combined['Item_MRP_Bins'].value_counts())

=== Phase 2: Feature Engineering ===

2.1 Creating basic derived features...
Outlet age range: 4 to 28 years
MRP distribution across bins:
Item_MRP_Bins
High        2434
Medium      2210
Low         1682
Premium     1440
Very_Low     757
Name: count, dtype: int64
